In [5]:
import numpy as np
import pandas as pd
import gc
import warnings
import os
import math
import catboost as cbt
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,precision_score, recall_score, f1_score,classification_report
from sklearn.model_selection import StratifiedKFold,GridSearchCV,cross_val_score,RandomizedSearchCV,train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime,timedelta
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn import datasets
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [6]:
train = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\train_set\first_round_training_data - 副本.csv')
test = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\test_set\first_round_testing_data - 副本.csv')
submit = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\submit_example.csv')

In [3]:
quality_map = {'Excellent': 0, 'Good': 1, 'Pass': 2, 'Fail': 3}
train['label'] = train['Quality_label'].map(quality_map)
data = pd.concat([train, test], ignore_index=True)

In [4]:
feature_name = ['Parameter{0}'.format(i) for i in range(5,11)]#选取特征
tr_index = ~data['label'].isnull()#将data中label这一列不为空的行数选取出来
X_train = data[tr_index][feature_name].reset_index(drop=True)#用tr_index选出trainset
#y = data[tr_index][bin_label].reset_index(drop=True).astype(int)#label of trainset
y = data[tr_index]['label'].reset_index(drop=True).astype(int)
X_test = data[~tr_index][feature_name].reset_index(drop=True)#用tr_index的取反得到testset

oof1 = np.zeros((X_train.shape[0],4))
prediction1 = np.zeros((X_test.shape[0],4))

#train_x, test_x, train_y, test_y = train_test_split(X_train, y)
pipeline = Pipeline([('clf', RandomForestClassifier(criterion='entropy',random_state=2019,
                    class_weight ='balanced'))])

n_estimators = range(1000,2000,100)
# 最大深度
max_depth = [5,6,7,8,9]
# 切分一个节点最小数量
min_samples_split = [5,7,9,11,13]
# 每一叶子节点最小数量
min_samples_leaf = [2,4,6,8,10]
# Method of selecting samples for training each tree
bootstrap = [True]

random_grid = {
                'clf__n_estimators': n_estimators,
                'clf__max_depth': max_depth,
                'clf__min_samples_split': min_samples_split,
                'clf__min_samples_leaf': min_samples_leaf,
                'clf__bootstrap': bootstrap,
               }
grid_search = GridSearchCV(pipeline,random_grid, n_jobs=-1, verbose=1, scoring=None)
grid_search.fit(X_train, y)
print('最佳效果：%0.3f' % grid_search.best_score_)
print('最优参数：')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(random_grid.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
oof1 = grid_search.predict_proba(X_train)
prediction1 = grid_search.predict(X_test)
print('logloss',log_loss(pd.get_dummies(y).values, oof1))
print('ac',accuracy_score(y, np.argmax(oof1,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y] - oof1))/480))
beta=2
p1=precision_score(y,np.argmax(oof1,axis=1),average='micro')
r1=recall_score(y,np.argmax(oof1,axis=1),average='micro')
p2=precision_score(y,np.argmax(oof1,axis=1),average='macro')
r2=recall_score(y,np.argmax(oof1,axis=1),average='macro')
print('micro precision',p1)
print('micro recall',r1)
print('macro precision',p2)
print('macro recall',r2)
print('micro F2-Score',(1+beta*beta)*p1*r1/(beta*beta*p1+r1))
print('macro F2-Score',(1+beta*beta)*p2*r2/(beta*beta*p2+r2))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  8.9min finished


最佳效果：0.444
最优参数：
	clf__bootstrap: True
	clf__n_estimators: 20000
logloss 0.4608030360105214
ac 0.8056666666666666
mae 0.11129247351004659
micro precision 0.8056666666666666
micro recall 0.8056666666666666
macro precision 0.7883654762144149
macro recall 0.8164407214020464
micro F2-Score 0.8056666666666666
macro F2-Score 0.8106668333265786


In [6]:
train['Predict_label1'] = np.argmax(prediction1,axis=1)

,Predict_label,label,Quality_label
0,3,2,Pass
1,3,3,Fail
2,3,3,Fail
3,3,3,Fail
4,3,3,Fail
5,3,1,Good
6,1,1,Good
7,1,1,Good
8,1,2,Pass
9,1,1,Good


In [5]:
oof2 = np.zeros((X_train.shape[0],4))
prediction2 = np.zeros((X_test.shape[0],4))

cbt_model = cbt.CatBoostClassifier(iterations=1500,loss_function='MultiClass',learning_rate=0.05,verbose=500,
                                   early_stopping_rounds=2300,task_type='GPU',)  
cbt_model.fit(X_train, y,eval_set=(X_train, y))

oof2 = cbt_model.predict_probo(X_train)
prediction2 = cbt_model.predict(X_test)

0:	learn: -1.3603531	test: -1.3603529	best: -1.3603529 (0)	total: 41.9ms	remaining: 1m 2s
500:	learn: -0.8820897	test: -0.8820896	best: -0.8820896 (500)	total: 6.33s	remaining: 12.6s
1000:	learn: -0.7309175	test: -0.7309173	best: -0.7309173 (1000)	total: 13.2s	remaining: 6.56s
1499:	learn: -0.6128376	test: -0.6128375	best: -0.6128375 (1499)	total: 20.1s	remaining: 0us
bestTest = -0.6128374837
bestIteration = 1499
Shrink model to first 1500 iterations.


In [ ]:
train['Predict_label2'] = np.argmax(prediction2,axis=1)

In [6]:
oof3 = np.zeros((X_train.shape[0],4))
prediction3 = np.zeros((X_test.shape[0],4))

lgb_model=LGBMClassifier(boosting_type="gbdt",num_leaves=23,reg_alpha=10, reg_lambda=5,max_depth=7,
                                 learning_rate=0.05, n_estimators=2000,colsample_bytree=0.7, subsample_freq=1,
                                 objective='multiclass', silent=True,subsample=0.7,min_child_samples=5,)
lgb_model.fit(X_train, y, eval_set=(X_train, y),early_stopping_rounds=1800, verbose=1000,)

oof3 = lgb_model.predict_proba(X_train)
prediction3 = lgb_model.predict(X_test)

Training until validation scores don't improve for 1800 rounds.
[1000]	training's multi_logloss: 0.844443
[2000]	training's multi_logloss: 0.747822
Did not meet early stopping. Best iteration is:
[2000]	training's multi_logloss: 0.747822


In [ ]:
train['Predict_label3'] = np.argmax(prediction3,axis=1)

In [7]:
oof4 = np.zeros((X_train.shape[0],4))
prediction4 = np.zeros((X_test.shape[0],4))

xgb_model = XGBClassifier(silent=True ,learning_rate= 0.05,min_child_weight=5,max_depth=7,gamma=0.1,
                      subsample=0.7,max_delta_step=0,colsample_bytree=0.7,reg_lambda=5,scale_pos_weight=1,
                      objective= 'multiclass',n_estimators=2000,seed=1000,verbose=1000)
xgb_model.fit(X_train, y)

oof4 = xgb_model.predict_proba(X_train)
prediction4 = xgb_model.predict(X_test)

In [ ]:
train['Predict_label4'] = np.argmax(prediction4,axis=1)

In [ ]:
train[['Predict_label1','Predict_label2','Predict_label3','Predict_label4','label']]

In [12]:
same1 = (prediction1.values == prediction2.values)
same2 = (prediction2.values == prediction3.values)
same3 = (prediction3.values == prediction4.values)
same4 = (same1 == same2)
same5 = (same2 == same3)
same = (same4 == same5)

In [ ]:
test['same'] = same